# 📘 Zestaw zadań — LangChain: Podstawowe komponenty + Chat z pamięcią

W tym notatniku znajdziesz *intuicyjne zadania* typu **uzupełnij kod**.
Zadania odnoszą się do notebooków z przykładami:
- **2_2_LangChain_core_components.ipynb**
- **2_4_LangChain_chat_with_memory.ipynb**

> **Wskazówka:** miejsca do uzupełnienia oznaczono jako `# TODO` lub `...`.


## Zadanie 1 — `PromptTemplate`: zdefiniuj szablon i wyrenderuj komunikat

In [ ]:
# Cel: Uzupełnij definicję PromptTemplate oraz wyrenderuj gotowy prompt.
# Odniesienie: 2_2_LangChain_core_components.ipynb

from langchain_core.prompts import PromptTemplate

# 1) Zdefiniuj zmienne wejściowe szablonu (powinny zawierać 'topic' i 'tone')
input_variables = ["topic", "tone"]  # TODO: możesz pozostawić jak jest lub zmodyfikować

# 2) Zdefiniuj treść szablonu tak, aby używać {topic} i {tone}
template = """Napisz krótki akapit o temacie: {topic}.
Utrzymaj ton wypowiedzi: {tone}."""  # TODO: dostosuj tekst jeśli chcesz

prompt = PromptTemplate(input_variables=input_variables, template=template)

# 3) Wyrenderuj gotowy prompt podstawiając wartości
rendered = prompt.format(topic="uczenie maszynowe", tone="entuzjastyczny")  # TODO: zmień wartości

print(rendered)
assert "{topic}" not in rendered and "{tone}" not in rendered, "Wygląda na to, że nie sformatowałeś szablonu."

## Zadanie 2 — `ChatPromptTemplate` i `MessagesPlaceholder` (historia rozmowy)

In [ ]:
# Cel: Zbuduj ChatPromptTemplate z miejscem na historię rozmowy.
# Odniesienie: 2_4_LangChain_chat_with_memory.ipynb

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 1) Uzupełnij szablon wiadomości: system, historia (placeholder), user
prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle po polsku."),  # TODO: możesz zmienić reguły
    MessagesPlaceholder(variable_name="history"),  # TODO: pozostaw jako placeholder
    ("user", "{input}"),  # TODO: nie zmieniaj klucza 'input'
])

# 2) Zasymuluj wypełnienie: podstaw 'history' i 'input' (bez wywołania modelu)
fake_history = [
    {"role": "user", "content": "Cześć, jak działa pamięć w czacie?"},
    {"role": "assistant", "content": "Pozwala wziąć pod uwagę poprzednie wiadomości."},
]

rendered = prompt.format_messages(history=fake_history, input="A jak zapisać ją w LangChain?")
for m in rendered:
    print(m.type if hasattr(m, "type") else m["role"], ":", m.content if hasattr(m, "content") else m["content"])

## Zadanie 3 — `RunnableWithMessageHistory`: opakuj łańcuch o pamięć

In [ ]:
# Cel: Zbuduj prosty łańcuch z pamięcią konwersacji.
# Odniesienie: 2_4_LangChain_chat_with_memory.ipynb

import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import InMemoryChatMessageHistory

# Upewnij się, że masz ustawione OPENAI_API_KEY w środowisku.
# os.environ["OPENAI_API_KEY"] = "sk-..."  # NIE wstawiaj klucza do repo

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # TODO: możesz zmienić model

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś asystentem. Pamiętaj kontekst rozmowy."),
    MessagesPlaceholder("history"),
    ("user", "{input}"),
])

chain = prompt | llm  # TODO: prosta kompozycja

store = {}
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
)

# (Nie wywołujemy realnie modelu w testach); Sprawdź, czy obiekt istnieje.
assert chain_with_memory is not None, "Nie udało się zbudować łańcucha z pamięcią."

## Zadanie 4 — `OutputParser`: wyodrębnij czysty tekst

In [ ]:
# Cel: Dodaj prosty parser, który wymusi zwrot zwykłego ciągu tekstowego.
# Odniesienie: 2_2_LangChain_core_components.ipynb

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

template = "Podaj definicję pojęcia: {term} w jednym zdaniu."
prompt = PromptTemplate.from_template(template)

# TODO: Złóż łańcuch prompt -> llm -> parser
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

chain = prompt | llm | parser  # TODO: ta linia jest kluczowa

# Nie wykonujemy wywołania. Sprawdźmy typ ostatniego elementu łańcucha:
assert parser is not None, "Parser nie został utworzony."

## Zadanie 5 — Pamięć rozmowy: dodaj bufor rozmowy do łańcucha
*Uwaga:* interfejs pamięci w LangChain ewoluuje. Poniższy szkic pokazuje ideę z buforem rozmów.

In [ ]:
# Cel: Zademonstruj integrację bufora historii z łańcuchem (wersja poglądowa).
# Jeśli Twoja wersja LangChain oferuje nowsze podejście, użyj go.

try:
    from langchain.memory import ConversationBufferMemory  # API może się różnić w wersjach
    memory = ConversationBufferMemory(return_messages=True)
    memory.save_context({"input": "Hej!"}, {"output": "Cześć, w czym pomóc?"})
    hist = memory.load_memory_variables({})
    print("Historia:", hist)
except Exception as e:
    print("Pominięto demonstrację ConversationBufferMemory (inne API?):", e)